CS4001/4042 Assignment 1
---
Part B, Q1 (15 marks)
---

Real world datasets often have a mix of numeric and categorical features – this dataset is one example. To build models on such data, categorical features have to be encoded or embedded.

PyTorch Tabular is a library that makes it very convenient to build neural networks for tabular data. It is built on top of PyTorch Lightning, which abstracts away boilerplate model training code and makes it easy to integrate other tools, e.g. TensorBoard for experiment tracking.

For questions B1 and B2, the following features should be used:   
- **Numeric / Continuous** features: dist_to_nearest_stn, dist_to_dhoby, degree_centrality, eigenvector_centrality, remaining_lease_years, floor_area_sqm
- **Categorical** features: month, town, flat_model_type, storey_range



---



In [1]:
# !pip install pytorch_tabular[extra]

In [2]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

import torch
import torch.nn as nn

from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

In [3]:
# Adding this to deal with the following warnings
# FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through
# chained assignment using an inplace method. The behavior will change in pandas 3.0.
# This inplace method will never work because the intermediate object on which we are
# setting values always behaves as a copy. For example, when doing
# 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)'
# or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.
import warnings
warnings.filterwarnings('ignore')

> Divide the dataset (‘hdb_price_prediction.csv’) into train and test sets by using entries from year 2020 and before as training data, and year 2021 as test data (validation set is not required).
**Do not** use data from year 2022 and year 2023.



In [4]:
df = pd.read_csv('hdb_price_prediction.csv')

# TODO: Enter your code here
# Filter out rows for years 2022 and 2023
filtered_data = df[df['year'] <= 2021]

# Split into training and test sets
train_data = filtered_data[filtered_data['year'] <= 2020]
test_data = filtered_data[filtered_data['year'] == 2021]
print(train_data.columns)

Index(['month', 'year', 'town', 'full_address', 'nearest_stn',
       'dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality',
       'eigenvector_centrality', 'flat_model_type', 'remaining_lease_years',
       'floor_area_sqm', 'storey_range', 'resale_price'],
      dtype='object')


> Refer to the documentation of **PyTorch Tabular** and perform the following tasks: https://pytorch-tabular.readthedocs.io/en/latest/#usage
- Use **[DataConfig](https://pytorch-tabular.readthedocs.io/en/latest/data/)** to define the target variable, as well as the names of the continuous and categorical variables.
- Use **[TrainerConfig](https://pytorch-tabular.readthedocs.io/en/latest/training/)** to automatically tune the learning rate. Set batch_size to be 1024 and set max_epoch as 50.
- Use **[CategoryEmbeddingModelConfig](https://pytorch-tabular.readthedocs.io/en/latest/models/#category-embedding-model)** to create a feedforward neural network with 1 hidden layer containing 50 neurons.
- Use **[OptimizerConfig](https://pytorch-tabular.readthedocs.io/en/latest/optimizer/)** to choose Adam optimiser. There is no need to set the learning rate (since it will be tuned automatically) nor scheduler.
- Use **[TabularModel](https://pytorch-tabular.readthedocs.io/en/latest/tabular_model/)** to initialise the model and put all the configs together.

In [5]:
# TODO: Enter your code here
data_config = DataConfig(
    target=["resale_price"],
    continuous_cols=["dist_to_nearest_stn", "dist_to_dhoby",
"degree_centrality", "eigenvector_centrality", "remaining_lease_years",
"floor_area_sqm"],
    categorical_cols=["month", "town", "flat_model_type", "storey_range"],
)

trainer_config = TrainerConfig(
    batch_size=1024,
    max_epochs=50,
)

model_config = CategoryEmbeddingModelConfig(
    task="regression",
    layers="50",  # Number of nodes in each layer
)

optimizer_config = OptimizerConfig() # Default is Adam

tabular_model = TabularModel(
    data_config=data_config,
    trainer_config=trainer_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
)


2025-03-11 14:20:33,141 - {pytorch_tabular.tabular_model:146} - INFO - Experiment Tracking is turned off

> Report the test RMSE error and the test R2 value that you obtained.



In [6]:
# TODO: Enter your code here
tabular_model.fit(train=train_data)
result = tabular_model.evaluate(test_data)
pred_df = tabular_model.predict(test_data)
tabular_model.save_model("examples/basic")
loaded_model = TabularModel.load_model("examples/basic")

Seed set to 42


2025-03-11 14:20:33,178 - {pytorch_tabular.tabular_model:548} - INFO - Preparing the DataLoaders

2025-03-11 14:20:33,227 - {pytorch_tabular.tabular_datamodule:522} - INFO - Setting up the datamodule for          
regression task

2025-03-11 14:20:33,351 - {pytorch_tabular.tabular_model:599} - INFO - Preparing the Model: CategoryEmbeddingModel

2025-03-11 14:20:33,392 - {pytorch_tabular.tabular_model:342} - INFO - Preparing the Trainer

...

Output()

`Trainer.fit` stopped: `max_epochs=50` reached.


2025-03-11 14:22:35,975 - {pytorch_tabular.tabular_model:689} - INFO - Training the model completed

2025-03-11 14:22:35,977 - {pytorch_tabular.tabular_model:1529} - INFO - Loading the best model

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL omegaconf.dictconfig.DictConfig was not an allowed global by default. Please use `torch.serialization.add_safe_globals([DictConfig])` or the `torch.serialization.safe_globals([DictConfig])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default

Ignore the error above, refer to https://github.com/pytorch/opacus/issues/690


In [8]:
result = tabular_model.evaluate(test_data)
pred_df = tabular_model.predict(test_data)
pred_df.head()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       67060428800.0       │
│  test_mean_squared_error  │       67060428800.0       │
└───────────────────────────┴───────────────────────────┘

,resale_price_prediction
87370,221761.890625
87371,229285.078125
87372,271750.125000
87373,272102.906250
87374,262278.906250


In [21]:
import math
# pip install torcheval
from torcheval.metrics.functional import r2_score
# Extract predicted values
predicted = torch.tensor(pred_df["resale_price_prediction"].values)

# Extract ground truth values
actual = torch.tensor(test_data["resale_price"].values)

# Compute R2 score and RMSE
r2 = r2_score(predicted, actual)
rmse = math.sqrt(result[0]['test_mean_squared_error'])
print("RMSE:", rmse)
print("R2 Score:", r2.item())


RMSE: 258960.28421362222
R2 Score: -1.5351788847008763


> Print out the corresponding rows in the dataframe for the top 25 test samples with the largest errors. 



In [22]:
# TODO: Enter your code here
predicted = pred_df["resale_price_prediction"].values
actual = test_data["resale_price"].values

# Calculate absolute errors
absolute_errors = np.abs(predicted - actual)

# Add the errors as a new column to the test_data DataFrame
test_data_with_errors = test_data.copy()
test_data_with_errors["absolute_error"] = absolute_errors

# Sort by absolute_error in descending order and get top 25 rows
top_25_errors = test_data_with_errors.sort_values(by="absolute_error", ascending=False).head(25)

# Print the corresponding rows
print(top_25_errors)

        month  year             town              full_address   nearest_stn  \
106199     12  2021       QUEENSTOWN            92 DAWSON ROAD    Queenstown   
90608      12  2021           BISHAN     273B BISHAN STREET 24        Bishan   
100836      6  2021  KALLANG/WHAMPOA          39 JALAN BAHAGIA     Boon Keng   
101237     11  2021  KALLANG/WHAMPOA          8 BOON KENG ROAD     Bendemeer   
93930      12  2021     CENTRAL AREA        1D CANTONMENT ROAD   Outram Park   
90483       9  2021           BISHAN     273A BISHAN STREET 24        Bishan   
93931      12  2021     CENTRAL AREA        1B CANTONMENT ROAD   Outram Park   
90432       8  2021           BISHAN     275A BISHAN STREET 24        Bishan   
101087      9  2021  KALLANG/WHAMPOA          46 JALAN BAHAGIA     Boon Keng   
112731      8  2021        TOA PAYOH  138A LORONG 1A TOA PAYOH      Braddell   
93904      11  2021     CENTRAL AREA        1C CANTONMENT ROAD   Outram Park   
93929      12  2021     CENTRAL AREA    

Part B, Q2 (10 marks)
---
In Question B1, we used the Category Embedding model. This creates a feedforward neural network in which the categorical features get learnable embeddings. In this question, we will make use of a library called Pytorch-WideDeep. This library makes it easy to work with multimodal deep-learning problems combining images, text, and tables. We will just be utilizing the deeptabular component of this library through the TabMlp network.

In [ ]:
# !pip install pytorch-widedeep

In [24]:
from pytorch_widedeep.preprocessing import TabPreprocessor
from pytorch_widedeep.models import TabMlp, WideDeep
from pytorch_widedeep import Trainer
from pytorch_widedeep.metrics import R2Score

>Divide the dataset (‘hdb_price_prediction.csv’) into train and test sets by using entries from the year 2020 and before as training data, and entries from 2021 and after as the test data（validation set is not required here).

In [ ]:
# TODO: Enter your code here
df = pd.read_csv('hdb_price_prediction.csv')
# Split into training and test sets
train_data = df[df['year'] <= 2020]
test_data = df[df['year'] >= 2021]
# print(train_data.shape)
# print(test_data.shape)

(87370, 14)
(72183, 14)


>Refer to the documentation of Pytorch-WideDeep and perform the following tasks:
https://pytorch-widedeep.readthedocs.io/en/latest/index.html
* Use [**TabPreprocessor**](https://pytorch-widedeep.readthedocs.io/en/latest/examples/01_preprocessors_and_utils.html#2-tabpreprocessor) to create the deeptabular component using the continuous
features and the categorical features. Use this component to transform the training dataset.
* Create the [**TabMlp**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/model_components.html#pytorch_widedeep.models.tabular.mlp.tab_mlp.TabMlp) model with 2 hidden layers in the MLP, with 200 and 100 neurons respectively.
* Create a [**Trainer**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/trainer.html#pytorch_widedeep.training.Trainer) for the training of the created TabMlp model with the root mean squared error (RMSE) cost function. Train the model for 60 epochs using this trainer, keeping a batch size of 64. (Note: set the *num_workers* parameter to 0.)

In [27]:
# TODO: Enter your code here
# Define categorical and continuous columns
continuous_cols = ["dist_to_nearest_stn", "dist_to_dhoby",
"degree_centrality", "eigenvector_centrality", "remaining_lease_years",
"floor_area_sqm"]  # No need to specify embed_dim explicitly
cat_embed_cols = ["month", "town", "flat_model_type", "storey_range"]

# Initialize TabPreprocessor without specifying embed_dim
tab_preprocessor = TabPreprocessor(
    continuous_cols=continuous_cols,
    cat_embed_cols=cat_embed_cols,
    cols_to_scale = continuous_cols)

# Fit and transform the training data
X_tab = tab_preprocessor.fit_transform(train_data)

# Inspect the automatically computed embedding dimensions
print(tab_preprocessor.cat_embed_input)

[('month', 12, 6), ('town', 26, 10), ('flat_model_type', 43, 13), ('storey_range', 17, 8)]


In [28]:
X_tab

array([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
        -1.45273689e-02, -1.03556571e+00, -2.21217732e+00],
       [ 1.00000000e+00,  1.00000000e+00,  2.00000000e+00, ...,
        -1.45273689e-02, -1.09350058e+00, -1.26511034e+00],
       [ 1.00000000e+00,  1.00000000e+00,  2.00000000e+00, ...,
        -1.96827730e-01, -9.51882003e-01, -1.26511034e+00],
       ...,
       [ 1.20000000e+01,  2.60000000e+01,  8.00000000e+00, ...,
        -2.68613872e-01, -6.75082048e-01,  1.82315157e+00],
       [ 1.20000000e+01,  2.60000000e+01,  2.00000000e+01, ...,
        -2.68613872e-01, -6.62207631e-01,  1.98785887e+00],
       [ 1.20000000e+01,  2.60000000e+01,  8.00000000e+00, ...,
        -2.68613872e-01, -6.81519256e-01,  1.82315157e+00]])

>Report the test RMSE and the test R2 value that you obtained.

In [ ]:
# TODO: Enter your code here

Part B, Q3 (10 marks)
---
Besides ensuring that your neural network performs well, it is important to be able to explain the model’s decision. **Captum** is a very handy library that helps you to do so for PyTorch models.

Many model explainability algorithms for deep learning models are available in Captum. These algorithms are often used to generate an attribution score for each feature. Features with larger scores are more ‘important’ and some algorithms also provide information about directionality (i.e. a feature with very negative attribution scores means the larger the value of that feature, the lower the value of the output).

In general, these algorithms can be grouped into two paradigms:
- **perturbation based approaches** (e.g. Feature Ablation)
- **gradient / backpropagation based approaches** (e.g. Saliency)

The former adopts a brute-force approach of removing / permuting features one by one and does not scale up well. The latter depends on gradients and they can be computed relatively quickly. But unlike how backpropagation computes gradients with respect to weights, gradients here are computed **with respect to the input**. This gives us a sense of how much a change in the input affects the model’s outputs.




---



In [ ]:
!pip install captum

In [ ]:
from captum.attr import Saliency, InputXGradient, IntegratedGradients, GradientShap, FeatureAblation

> First, use the train set (year 2020 and before) and test set (year 2021) following the splits in Question B1 (validation set is not required here). To keep things simple, we will **limit our analysis to numeric / continuous features only**. Drop all categorical features from the dataframes. Standardise the features via **StandardScaler** (fit to training set, then transform all).

In [ ]:
# TODO: Enter your code here

> Follow this tutorial to generate the plot from various model explainability algorithms (https://captum.ai/tutorials/House_Prices_Regression_Interpret).
Specifically, make the following changes:
- Use a feedforward neural network with 3 hidden layers, each having 5 neurons. Train using Adam optimiser with learning rate of 0.001.
- Use Input x Gradients, Integrated Gradients, DeepLift, GradientSHAP, Feature Ablation. To avoid long running time, you can limit the analysis to the first 1000 samples in test set.

In [ ]:
# TODO: Enter your code here

> Read the following [descriptions](https://captum.ai/docs/attribution_algorithms) and [comparisons](https://captum.ai/docs/algorithms_comparison_matrix) in Captum to build up your understanding of the difference of various explainability algorithms. Based on your plot, identify the three most important features for regression. Explain how each of these features influences the regression outcome.


\# TODO: \<Enter your answer here\>

Part B, Q4 (10 marks)
---

Model degradation is a common issue faced when deploying machine learning models (including neural networks) in the real world. New data points could exhibit a different pattern from older data points due to factors such as changes in government policy or market sentiments. For instance, housing prices in Singapore have been increasing and the Singapore government has introduced 3 rounds of cooling measures over the past years (16 December 2021, 30 September 2022, 27 April 2023).

In such situations, the distribution of the new data points could differ from the original data distribution which the models were trained on. Recall that machine learning models often work with the assumption that the test distribution should be similar to train distribution. When this assumption is violated, model performance will be adversely impacted.  In the last part of this assignment, we will investigate to what extent model degradation has occurred.




---



In [ ]:
!pip install alibi-detect

In [ ]:
from alibi_detect.cd import TabularDrift

> Evaluate your model from B1 on data from year 2022 and report the test R2.

In [ ]:
# TODO: Enter your code here

> Evaluate your model from B1 on data from year 2023 and report the test R2.

In [ ]:
# TODO: Enter your code here

> Did model degradation occur for the deep learning model?

\# TODO: \<Enter your answer here\>

Model degradation could be caused by [various data distribution shifts](https://huyenchip.com/2022/02/07/data-distribution-shifts-and-monitoring.html#data-shift-types): covariate shift (features), label shift and/or concept drift (altered relationship between features and labels).
There are various conflicting terminologies in the [literature](https://www.sciencedirect.com/science/article/pii/S0950705122002854#tbl1). Let’s stick to this reference for this assignment.

> Using the **Alibi Detect** library, apply the **TabularDrift** function with the training data (year 2020 and before) used as the reference and **detect which features have drifted** in the 2023 test dataset. Before running the statistical tests, ensure you **sample 1000 data points** each from the train and test data. Do not use the whole train/test data. (Hint: use this example as a guide https://docs.seldon.io/projects/alibi-detect/en/stable/examples/cd_chi2ks_adult.html)


In [ ]:
# TODO: Enter your code here

> Assuming that the flurry of housing measures have made an impact on the relationship between all the features and resale_price (i.e. P(Y|X) changes), which type of data distribution shift possibly led to model degradation?

\# TODO: \<Enter your answer here\>

> From your analysis via TabularDrift, which features contribute to this shift?

\# TODO: \<Enter your answer here\>

> Suggest 1 way to address model degradation and implement it, showing improved test R2 for year 2023.

\# TODO: \<Enter your answer here\>

In [ ]:
# TODO: Enter your code here